## This notebook is designed as an addition to the lecture on transport phenomena (29.10.2020)

This notebook serves as an addition to the transport phenomena lab. It contains more examples in order to gain a deeper understanding. __You do not need to change anything in the code in order for it to run.__ However, feel free to change parameters to see what happens. The three examples describe:

- A central round isothermal heat  

- A constant heat input on the left (as opposed to a isothermal left side)

- A isothermal left side at $T_{hot}$ and a isothermal right side at $T_{right}$

While in all of these cases the transportion mechanism is the same as before, the contraints are very different.

You can run the cells either by clicking on the cell and then on Run or press ctrl+enter


## 0. Install packages

In [1]:
!pip install numpy
!pip install matplotlib

## 1. Helper Functions


In [10]:
# helper functions for animation
def KtoC(u):
    # converts temperature funcion from Kelvin to Celsius
    conversionfactor = 273.15
    return (u-conversionfactor)

def init():
    # initializes figure and sets desgin of image
    global fig, ax, im
    fig = plt.figure(1)
    # image with cmap and color bar
    im = plt.imshow(KtoC(u_tot[:,:,0]),cmap=plt.get_cmap('hot'), vmin=KtoC(T_initial),vmax=KtoC(T_hot),extent=[0,w,h,0])
    plt.xlabel('[cm]')
    plt.ylabel('[cm]')
    cbar_ax = fig.add_axes([0.9, 0.15, 0.03, 0.7]) 
    cbar_ax.set_xlabel('$T$ / °C', labelpad=20)
    fig.colorbar(im, cax=cbar_ax)
    #initialize image with temperature at time 0
    im.set_data(u_tot[:,:,0])
    plt.close()
    return 

def animate(i):
    # changes image of temperature to time: i*dt 
    global fig, u_tot
    fig.suptitle('{:.1f} min'.format(simulation_time/nframes*i))  #set title above image to current time
    im.set_data(KtoC(u_tot[:,:,i])) #change temperature to i-th temperature
    return 

# helper function to propagate through time
def do_timestep(u, dnframes):
    for i in range(dnframes):
        u[1:-1, 1:-1] = u[1:-1, 1:-1] + alpha * dt * (
              (u[2:, 1:-1] - 2*u[1:-1, 1:-1] + u[:-2, 1:-1])/dx2
              + (u[1:-1, 2:] - 2*u[1:-1, 1:-1] + u[1:-1, :-2])/dy2 )
        u[0,:] = u[1,:]    # set edge cases to same as neighbour
        u[-1,:] = u[-2,:]  # set edge cases to same as neighbour
        u[:,0] = u[:,1]    # set edge cases to same as neighbour
        u[:,-1] = u[:,-2]  # set edge cases to same as neighbour
        u = get_constraints(u.copy()) #implement constraints
    return u


## 2. Paramter 

In [11]:
### physical parameter ###

#geometries
w = 10       # width [mm]
h = 10       # height [mm]

# temperatures
T_initial=300  # inital temperature[K]
T_hot = 700   # hot temperature [K]
T_max = 1400 # maximum temperature [K]
Q = 10**5 # Radiogenic heat production [W/m³] (for part c)

#material constants
alpha = 10*10**-6  # heat transfer coefficient [m²/s]
c_heat = 460 #[J/kg K] Heat Capacity of Steel
rho = 8050 # Density of Steel [kg/m³]

### simulation parameter ###

#time
simulation_time = 60 #[min] 

#simulation resolution
dx = 0.1      # finite step in x direction [mm]
dy = 0.1     #finite step in y direction [mm]

#number of nodes
nx = int(w/dx) #number of nodes in x direction 
ny = int(h/dy) #number of nodes in y direction

dx2, dy2 = dx*dx, dy*dy  #[mm²] Helper Variables
dt = dx2 * dy2 / (2 * alpha * (dx2 + dy2)) #[ms]

# Number of frames 
nframes = 40 #to keep computation feasable, only 40 frames are used, regardeless simulation time
dnframes = int(nsteps/nframes) #number of timesteps in a single frame

## 3a. Central round isothermal heat 

### In this addition, instead of a linear isothermal heat source on the left, a circular isothermal heat source in the middle of a plate is used

Mathematically, this constraint for the temperature function $u(x,y,t)$ can be formulated as:

$ u(x,y,t) = T_{hot}$ for all $(x-cx)²+(y-cy)² \leq r $

Where cx, cy are the respective coordinates of circle's focus and r is the circle's radius. The other constraint stays the same:

$ u(x,y,t=0) = T_{cool}$ for all $(x-cx)²+(y-cy)² > r $

In [12]:
# calculate each temperature(x,y,t) with central difference in space and Euler forward in time

def get_constraints(u):
    r = 2 # radius in [mm]
    cx = 5 # x position of center [mm]
    cy = 5 # y position of center [mm]
    # definition of central circle
    for i in range(nx): #loop through all cooridantes to define circle of heat
        for j in range(ny):
            h2 = (i*dx-cx)**2 + (j*dy-cy)**2 #direct distance to center of heat
            if(h2<r**2):
                u[i,j] = T_hot
    return u
            
    

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# time
simulation_time = 10 #[min] 
nsteps = int(simulation_time*60/dt*1000) # Number of timesteps
dnframes = int(nsteps/nframes) #number of timesteps in a single frame


### Initialization ###

# initialization at time 0
u0 = T_initial * np.ones((nx, ny)) # initialize temperature function to all Initial temperature
u0 = get_constraints(u0) #introduce constraints 

# initialization temperature function for all timesteps
u_tot = np.ones([nx,ny,nframes+1]) # initialize all to one (carried out to save time and allocate memory beforehand)
u_tot[:,:,0] = u0 # at timestep zero, equal to initialial temperature function 


### Simulation ###
#iterate through time
for i in range(nframes):
    global u
    u = u_tot[:,:,i] # current temperature function 
    u_tot[:,:,i+1]= do_timestep(u.copy(), dnframes) 

#execute animation
plt.ion()
init()
ani = animation.FuncAnimation(fig, animate, frames = nframes) 
HTML(ani.to_jshtml())

## 3b. Heat on the left with constant heat input 

### In this addition, instead of a linear isothermal heat source on the left, a constant heat input on the left is assumed

The initial condition as well as the adiabatic assumption stay the same. 
Initial condition:
- $ u(x\neq 0,y,t=0) = T_{initial} $ : At time $t=0$, the rest of the plat is cold

The plate is assumed to be adiabatic. Hence, the temperature function's change on the edges of the plate is zero. This can be formulated into the following boundary conditions:

Boundary conditions:
- $ \frac{\partial{u(x,y,t)}}{\partial{x}} = 0 \   $ for $ x=0 \ or \ x=w \ $ (w = plate's width) 

- $ \frac{\partial{u(x,y,t)}}{\partial{y}} = 0 \   $for  $ y=0 \ or \ x=h \ $ (h = plate's height) 


In [6]:
# calculate each temperature(x,y,t) with central difference in space and Euler forward in time
# calculate each temperature(x,y,t) with central difference in space and Euler forward in time
def get_constraints(u):
    u[:,0] = u[:,0] + Q*dt/(rho*c_heat)
    return u
            
    

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# time
simulation_time = 100 #[min] 
nsteps = int(simulation_time*60/dt*1000) # Number of timesteps
dnframes = int(nsteps/nframes) #number of timesteps in a single frame


### Initialization ###

# initialization at time 0
u0 = T_initial * np.ones((nx, ny)) # initialize temperature function to initial temperature
u0 = get_constraints(u0) #introduce constraints 

# initialization temperature function for all timesteps
u_tot = np.ones([nx,ny,nframes+1]) # initialize all to one (carried out to save time and allocate memory beforehand)
u_tot[:,:,0] = u0 # at timestep zero, equal to initialial temperature function 


### Simulation ###
#iterate through time
for i in range(nframes):
    global u
    u = u_tot[:,:,i] # current temperature function 
    u_tot[:,:,i+1]= do_timestep(u.copy(), dnframes) 

#execute animation
plt.ion()
init()
ani = animation.FuncAnimation(fig, animate, frames = nframes) 
HTML(ani.to_jshtml())

As you can see when running this animation, the plate's temperature profile will rise infinitely. That is because the heat input on the left will not stop and the heat is stored in the plate, leading to rising temperatures. 

## 3c. Isothermic hot temperature $ T_{hot}$ left, isothermic temperature  right  $ T_{initial}$

### In this addition, not just a isothermal temperature on the left is defined, but on the right as well


Mathematically, this constraint for the temperature function $u(x,y,t)$ can be formulated as:

$ u(x=0,y,t) = T_{hot} \ \ and \ \ u(x=w,y,t) = T_{right} = T_{initial}$

Where w is the width of the plate. The initial condition as well as the boundary conditions stay the same:

Initial condition:
- $ u(x\neq \ (0,w) \ ,y,t=0) = T_{initial} $ : At time $t=0$, the rest of the plat is cold

The plate is assumed to be adiabatic. Hence, the temperature function's change on the edges of the plate is zero. This can be formulated into the following boundary conditions:

Boundary conditions:
- $ \frac{\partial{u(x,y,t)}}{\partial{x}} = 0 \   $ for $ x=0 \ or \ x=w \ $ (w = plate's width) 

- $ \frac{\partial{u(x,y,t)}}{\partial{y}} = 0 \   $for  $ y=0 \ or \ x=h \ $ (h = plate's height) 


In [20]:
# calculate each temperature(x,y,t) with central difference in space and Euler forward in time
def get_constraints(u):
    u[:,0] = T_hot #left side is isothermal at T_hot
    u[:,-1:] = T_initial #right side is isothermal at T_initial
    return u
            
    

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# time
simulation_time = 60 #[min] 
nsteps = int(simulation_time*60/dt*1000) # Number of timesteps
dnframes = int(nsteps/nframes) #number of timesteps in a single frame


### Initialization ###

# initialization at time 0
u0 = T_initial * np.ones((nx, ny)) # initialize temperature function to initial temperature
u0 = get_constraints(u0) #introduce constraints 

# initialization temperature function for all timesteps
u_tot = np.ones([nx,ny,nframes+1]) # initialize all to one (carried out to save time and allocate memory beforehand)
u_tot[:,:,0] = u0 # at timestep zero, equal to initialial temperature function 


### Simulation ###
#iterate through time
for i in range(nframes):
    global u
    u = u_tot[:,:,i] # current temperature function 
    u_tot[:,:,i+1]= do_timestep(u.copy(), dnframes) 

#execute animation
plt.ion()
init()
ani = animation.FuncAnimation(fig, animate, frames = nframes) 
HTML(ani.to_jshtml())

As you can see when running this animation, the plate's temperature profile will not rise infinitely. Instead, it approaches a stationary state. With the hot side on the left and the cold side on the right, the temperature profile inbetween will eventually become linear. 